## Cosine Similarity Calculations
Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. Similarity measures have a multiude of uses in machine learning projects; they come in handy when matching strings, measuring distance, and extracting features. This similarity measurement is particularly concerned with orientation, rather than magnitude. 
In this case study, you'll use the cosine similarity to compare both a numeric data within a plane and a text dataset for string matching.

Load the Python modules, including cosine_similarity, from sklearn.metrics.pairwise

In [1]:
from google.colab import files
upload=files.upload()

Saving distance_dataset.csv to distance_dataset.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
plt.style.use('ggplot')
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity

**<font color='teal'> Load the distance dataset into a dataframe. </font>**

In [3]:
distance_data = pd.read_csv('/content/distance_dataset.csv')
#df = df.drop(columns='Unnamed: 0', axis=1)
distance_data.head()

,Unnamed: 0,X,Y,Z,ClusterID
0,0,5.135779,4.167542,5.787635,4
1,1,4.280721,5.770909,6.091044,4
2,2,8.329098,7.540436,3.247239,2
3,3,5.470224,5.069249,5.768313,4
4,4,2.381797,2.402374,3.879101,1


### Cosine Similarity with clusters and numeric matrices

All points in our dataset can be thought of as feature vectors. We illustrate it here as we display the __Cosine Similarity__ between each feature vector in the YZ plane and the [5, 5] vector we chose as reference. The sklearn.metrics.pairwise module provides an efficient way to compute the __cosine_similarity__ for large arrays from which we can compute the similarity.

 **<font color='teal'> First, create a 2D and a 3D matrix from the dataframe. The 2D matrix should contain the 'Y' and 'Z' columns and the 3D matrix should contain the 'X','Y', and 'Z' columns.</font>**

In [4]:
matrix_YZ = distance_data[['Y', 'Z']].to_numpy()        #to_numpy() to convert Pandas dataframe to Numpy array
matrix_XYZ = distance_data[['X', 'Y', 'Z']].to_numpy()

Calculate the cosine similarity for those matrices with reference planes of 5,5 and 5,5,5. Then subtract those measures from 1 in new features.

In [5]:
simCosine3D = 1. - cosine_similarity(matrix_XYZ, [[5,5,5]], 'cosine')
simCosine = 1. - cosine_similarity(matrix_YZ, [[5,5]], 'cosine')

Using the 2D matrix and the reference plane of (5,5) we can use a scatter plot to view the way the similarity is calculated using the Cosine angle.

In [12]:
figCosine = plt.figure(figsize=[10,8])

plt.scatter(distance_data.Y, distance_data.Z, c=simCosine[:,0], s=20)
plt.plot([0,5],[0,5], '--', color='dimgray')
plt.plot([0,3],[0,7.2], '--', color='dimgray')
plt.text(0.7,2.6,r'$\theta$ = 22.4 deg.', rotation=47, size=14)
plt.ylim([0,10])  # to set y-limit
plt.xlim([0,10])  # to set x-limit
plt.xlabel('Y', size=14)
plt.ylabel('Z', size=14)
plt.title('2D Cosine Similarity - YZ', size = 24)
cb = plt.colorbar()
cb.set_label('Similarity with (5,5)', size=14)

figCosine.savefig('similarity-cosine-2D.png')

Now, plot the 3D matrix with the similarity and the reference plane, (5,5,5).

In [13]:
from mpl_toolkits.mplot3d import Axes3D
figCosine3D = plt.figure(figsize=(10, 8))
ax = figCosine3D.add_subplot(111, projection='3d')

p = ax.scatter(matrix_XYZ[:,0], matrix_XYZ[:,1], matrix_XYZ[:,2], c=simCosine3D[:,0])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('3D Cosine Similarity - XYZ', size = 24)
cb = figCosine3D.colorbar(p)
cb.set_label('Similarity with (5,5,5)', size=14)
                                   
figCosine3D.tight_layout()
figCosine3D.savefig('cosine-3D.png', dpi=300, transparent=True)

----

### Cosine Similarity with text data
This is a quick example of how you can use Cosine Similarity to compare different text values or names for record matching or other natural language proecessing needs. 
First, we use count vectorizer to create a vector for each unique word in our Document 0 and Document 1. 

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
Document1 = "Starbucks Coffee"
Document2 = "Essence of Coffee"

corpus = [Document1,Document2]

X_train_counts = count_vect.fit_transform(corpus)

pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names(),index=['Document 0','Document 1'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,coffee,essence,of,starbucks
Document 0,1,0,0,1
Document 1,1,1,1,0


Now, we use a common frequency tool called TF-IDF to convert the vectors to unique measures.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=['Document 0','Document 1'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,coffee,essence,of,starbucks
Document 0,0.579739,0.000000,0.000000,0.814802
Document 1,0.449436,0.631667,0.631667,0.000000


Here, we finally apply the __Cosine Similarity__ measure to calculate how similar Document 0 is compared to any other document in the corpus. Therefore, the first value of 1 is showing that the Document 0 is 100% similar to Document 0 and 0.26055576 is the similarity measure between Document 0 and Document 1.

In [19]:
cosine_similarity(trsfm[0:1], trsfm)

array([[1.        , 0.26055567]])

Replace the current values for `Document 0` and `Document 1` with your own sentence or paragraph and apply the same steps as we did in the above example.

 **<font color='teal'> Combine the documents into a corpus.</font>**

In [25]:
count_vect_new = CountVectorizer()
qoute1 = 'It is really cold outside today'
qoute2 = 'The weather has been quite chilly these days'
corpus1 = [qoute1, qoute2]

 **<font color='teal'> Apply the count vectorizer to the corpus to transform it into vectors.</font>**

In [26]:
X_train_counts_new = count_vect_new.fit_transform(corpus1)

 **<font color='teal'> Convert the vector counts to a dataframe with Pandas.</font>**

In [30]:
pd.DataFrame(X_train_counts_new.toarray(), 
             columns = count_vect_new.get_feature_names(),
             index=['Quote 1','Quote 2'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,been,chilly,cold,days,has,is,it,outside,quite,really,the,these,today,weather
Quote 1,0,0,1,0,0,1,1,1,0,1,0,0,1,0
Quote 2,1,1,0,1,1,0,0,0,1,0,1,1,0,1


 **<font color='teal'> Apply TF-IDF to convert the vectors to unique frequency measures.</font>**

In [31]:
vectorizer_new = TfidfVectorizer()
trsfm_new = vectorizer_new.fit_transform(corpus1)
pd.DataFrame(trsfm_new.toarray(),
             columns = vectorizer_new.get_feature_names(),
             index=['Quote 1','Quote 2'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,been,chilly,cold,days,has,is,it,outside,quite,really,the,these,today,weather
Quote 1,0.000000,0.000000,0.408248,0.000000,0.000000,0.408248,0.408248,0.408248,0.000000,0.408248,0.000000,0.000000,0.408248,0.000000
Quote 2,0.353553,0.353553,0.000000,0.353553,0.353553,0.000000,0.000000,0.000000,0.353553,0.000000,0.353553,0.353553,0.000000,0.353553


 **<font color='teal'> Use the cosine similarity function to get measures of similarity for the sentences or paragraphs in your original document.</font>**

In [35]:
cosine_similarity(trsfm_new[0:1], trsfm_new)

array([[1., 0.]])